# Streaming Options

:::{note}
All the examples below require a live python process to experience the full interactivity of the plots.
:::

Options for handling live data streams:

```{eval-rst}
.. plotting-options-table:: Streaming Options
```

(option-backlog)=
## `backlog`

The `backlog` option allows to define the maximum number of rows to keep in the stream buffer when using a streaming data source. Default is `1000`. The object returned when `stream` is set is a HoloViews [`DynamicMap`](inv:holoviews#reference/containers/bokeh/DynamicMap).

In [ ]:
import hvplot.streamz  # noqa
from streamz.dataframe import Random

df = Random(interval='200ms', freq='50ms')
df.hvplot.table(width=400, backlog=5)

When running these two cells in a notebook one after the other, the data in the table above will be updated every 200ms for 2 seconds.

In [ ]:
import asyncio
await asyncio.sleep(2)
df.stop()

(option-stream)=
## `stream`

The `stream` option accepts a stream object for streaming plots, allowing data updates without re-rendering the entire plot. The object returned when `stream` is set is a HoloViews [`DynamicMap`](inv:holoviews#reference/containers/bokeh/DynamicMap).

In this first example, we set up and attach a HoloViews [`Pipe`](https://holoviews.org/user_guide/Streaming_Data.html#pipe) stream, we then call its `send` method in the next cell with a new DataFrame, which will entirely replace the plotted data, plotting a year after the other with an interval of 1 second.

In [ ]:
import hvplot.pandas  # noqa
from holoviews.streams import Pipe

df = hvplot.sampledata.stocks("pandas").set_index("date")
years = df.index.year.drop_duplicates()

sdf = df.loc[str(years[0]), :]
stream = Pipe(data=sdf)
plot_opts = dict(height=250, width=500, xlim=(df.index.min(), df.index.max()), ylim=(0, 6))
plot = sdf.hvplot.line(stream=stream, **plot_opts)
plot

In [ ]:
import time

for year in years[1:]:
    stream.send(df.loc[str(year), :])
    time.sleep(1)

In this second example we set up and attach a HoloViews [`Buffer`](https://holoviews.org/user_guide/Streaming_Data.html#buffer) stream, we then call its `send` method in the next cell with a new DataFrame, which will accumulate the data, gradually plotting the whole dataset with an interval of 1 second.

In [ ]:
import hvplot.pandas  # noqa
from holoviews.streams import Buffer

df = hvplot.sampledata.stocks("pandas").set_index("date")
years = df.index.year.drop_duplicates()

sdf = df.loc[str(years[0]), :]
stream = Buffer(data=sdf, index=False)
plot_opts = dict(height=250, width=500, xlim=(df.index.min(), df.index.max()), ylim=(0, 6))
plot = sdf.hvplot.line(stream=stream, **plot_opts)
plot

In [ ]:
import time

for year in years[1:]:
    stream.send(df.loc[str(year), :])
    time.sleep(1)